In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# -----------------------------
# Step 1: Load Data
# -----------------------------
adulterated = pd.read_csv("adulterated_watermelon_data.csv")
natural = pd.read_csv("natural_watermelon_data.csv")

adulterated["label"] = 1
natural["label"] = 0

data = pd.concat([adulterated, natural], axis=0).reset_index(drop=True)
X = data[["Chemical Shift (ppm)", "Intensity"]].values
y = data["label"].values.reshape(-1, 1)  # shape (n_samples, 1)

# -----------------------------
# Step 2: Train-Test Split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# -----------------------------
# Step 3: NumPy Neural Network
# -----------------------------
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return z * (1 - z)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)

input_neurons = X_train_scaled.shape[1]
hidden_neurons = 64
output_neurons = 1
epochs = 500
lr = 0.1

# Initialize weights
wh = np.random.randn(input_neurons, hidden_neurons) * 0.01
bh = np.zeros((1, hidden_neurons))
wout = np.random.randn(hidden_neurons, output_neurons) * 0.01
bout = np.zeros((1, output_neurons))

# Training NumPy NN
for i in range(epochs):
    # Forward
    hidden_input = np.dot(X_train_scaled, wh) + bh
    hidden_act = relu(hidden_input)
    output_input = np.dot(hidden_act, wout) + bout
    pred = sigmoid(output_input)

    # Backward
    error = y_train - pred
    d_output = error * sigmoid_derivative(pred)
    d_hidden = d_output.dot(wout.T) * relu_derivative(hidden_input)

    # Update
    wout += hidden_act.T.dot(d_output) * lr
    bout += np.sum(d_output, axis=0, keepdims=True) * lr
    wh += X_train_scaled.T.dot(d_hidden) * lr
    bh += np.sum(d_hidden, axis=0, keepdims=True) * lr

# Predictions
hidden_test = relu(np.dot(X_test_scaled, wh) + bh)
y_pred_numpy = np.round(sigmoid(np.dot(hidden_test, wout) + bout))

# -----------------------------
# Step 4: Keras Neural Network
# -----------------------------
model = Sequential([
    Dense(16, input_shape=(2,), activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=50, batch_size=8, verbose=0)

y_pred_prob_keras = model.predict(X_test_scaled)
y_pred_keras = (y_pred_prob_keras > 0.5).astype(int)

# -----------------------------
# Step 5: Comparison Table
# -----------------------------
print("\n--- NumPy NN Metrics ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_numpy):.4f}")
print(confusion_matrix(y_test, y_pred_numpy))
print(classification_report(y_test, y_pred_numpy))

print("\n--- Keras NN Metrics ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_keras):.4f}")
print(confusion_matrix(y_test, y_pred_keras))
print(classification_report(y_test, y_pred_keras))

print("\n--- Side by Side Predictions ---")
for i in range(len(y_test)):
    print(f"Actual: {y_test[i][0]}, NumPy NN: {y_pred_numpy[i][0]}, Keras NN: {y_pred_keras[i][0]}")


c:\projects\Watermelon\.venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

--- NumPy NN Metrics ---
Accuracy: 0.9800
[[25  0]
 [ 1 24]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        25
           1       1.00      0.96      0.98        25

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50


--- Keras NN Metrics ---
Accuracy: 0.9600
[[25  0]
 [ 2 23]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        25
           1       1.00      0.92      0.96        25

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50


--- Side by Side Predictions ---
Actual: 1, NumPy NN: 1.0, Keras NN: 1
Actual: 0, NumPy NN: 0.0, Keras NN: 0
Actual: 0, NumPy NN: 0.0, Keras NN: 0
Actual: 0, NumPy NN: 0.0, Keras NN: 